## Classic Strategy 

#### abstract class can present the logic relation between codes or modes. Then let visitors focus on method and describe of abstract classes instead instantiating.

In [5]:
!pip install abc

ERROR: Could not find a version that satisfies the requirement abc (from versions: none)
ERROR: No matching distribution found for abc


In [4]:
import abc

ModuleNotFoundError: No module named 'abs'

In [12]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

class LineItem:

    def __init__(self,product,quantity,price):
        self.product = product 
        self.quantity = quantity
        self.price = price 
    
    def total(self):
        return self.price*self.quantity 

class Order:

    def __init__(self,customer,cart,promotion=None):
        self.customer = customer 
        self.cart = list(cart)
        self.promotion = promotion 
    def total(self):
        if not hasattr(self,'__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total 
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self) #pass order to promotion function
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC): #the Strategy: an abstract base class

    @abstractmethod 
    def discount(self, order):
        """ Return discount as a positive dollar amount"""
    
class Fidelity_Promo(Promotion):
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self,order):
        return order.total()*0.5 if order.customer.fidelity >= 1000 else 0

class bulk_item_Promo(Promotion):
    def discount(self,order):
        """10% discount for each LineItem with 20 or more units"""
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

print(globals())


{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'from abc import ABC, abstractmethod\nfrom collections import namedtuple\n\nCustomer = namedtuple(\'Customer\',\'name fidelity\')\n\nclass LineItem:\n\n    def __init__(self,product,quantity,price):\n        self.product = product \n        self.quantity = quantity\n        self.price = price \n    \n    def total(self):\n        return self.price*self.quantity \n\nclass Order:\n\n    def __init__(self,customer,cart,promotion=None):\n        self.customer = customer \n        self.cart = list(cart)\n        self.promotion = promotion \n    def total(self):\n        if not hasattr(self,\'__total\'):\n            self.__total = sum(item.total() for item in self.cart)\n        return self.__total \n    \n    def due(self):\

In [2]:
joe = Customer('John Doe',0)
ann = Customer('Ann Simith',1100)
cart = [
    LineItem('banana',4,.5),
    LineItem('apple',10,1.5),
    LineItem('watermellon',5,5)
]
Order(joe,cart,Fidelity_Promo())
Order(ann,cart,Fidelity_Promo())

<Order total: 42.00 due: 21.00>

### Find the best strategy

In [15]:
promos = [globals()[name]() for name in globals() if name.endswith('_Promo') and name!= 'best_promo']
print(promos)
class best_promo(Promotion):
    def discount(self, order):
        """select best discount available"""
        #(print(promo.discount(order)) for promo in promos)
        return max(promo.discount(order) for promo in promos)

[<__main__.Fidelity_Promo object at 0x00000228BA583D90>, <__main__.bulk_item_Promo object at 0x00000228BA583E80>]


In [16]:
#Order(joe,cart,best_promo())
Order(ann,cart,best_promo())

<Order total: 42.00 due: 21.00>